# agents

> This module specifies Agent Classes.

In [ ]:
#| default_exp agents

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
class Agent:
    '''This Agent Class specifies agents  '''
    def __init__(self):
        self.location = None
        self.actons = None
        self.act = None
        self.energy = None
        
    def chooseAction(self):
        pass
        return self.act

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()